# Cálculo de autovalores y autovectores
Los métodos implementados son los siguientes:
- Método de la potencia: `potencia`.
- Método de la potencia inversa: `potencia_inversa`.
- Método de la potencia desplazada: `potencia_desplazada`.
- Método de la potencia inversa desplazada: `potencia_inversa_desplazada`.

In [1]:
from numpy import *
from numpy.linalg import eig, norm, solve

## Problema propuesto
Hallar información sobre los autovalores y autovectores de la matriz
$$A = \begin{pmatrix}
    -33 & 36 & -27 & 18 & -9 \\
    -33 & 51 & -45 & 30 & -15 \\
    -19 & 38 & -45 & 34 & -17 \\
    -9 & 18 & -27 & 27 & 15 \\
    -3 & 6 & -9 & 12 & -9
\end{pmatrix}.$$

In [2]:
A = array([
    [-33, 36, -27, 18, -9],
    [-33, 51, -45, 30, -15],
    [-19, 38, -45, 34, -17],
    [-9, 18, -27, 27, -15],
    [-3, 6, -9, 12, -9]
])

autovalores, autovectores = eig(A)
print("Autovalores\n", autovalores)
print("Autovectores\n", autovectores)

Autovalores
 [ 12.   6. -15.  -9.  -3.]
Autovectores
 [[ 4.08248290e-01 -7.67654634e-15 -8.94427191e-01  9.49240207e-15
   1.53647880e-15]
 [ 8.16496581e-01 -1.39299896e-14 -4.47213595e-01  4.08248290e-01
   2.89219539e-15]
 [ 4.08248290e-01 -4.08248290e-01 -1.08161513e-15  8.16496581e-01
   4.71990220e-15]
 [ 2.42114535e-16 -8.16496581e-01 -8.76481225e-16  4.08248290e-01
   4.47213595e-01]
 [ 9.77469767e-17 -4.08248290e-01 -3.72970734e-17 -1.32364172e-15
   8.94427191e-01]]


## Método de la potencia
Sea $A \in \mathcal{M}_n(\mathbb{C})$ y sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = Ax_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $A$.

In [3]:
def potencia(A, x0, iteraciones = 100):
    tolerancia = 1e-15
    x = x0
    
    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = A@x
        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia al problema para hallar el autovector asociado al autovalor de mayor módulo.

In [4]:
x0 = array([1, 0, 0, 0, 0])
x = potencia(A, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[ 8.94427191e-01  4.47213595e-01 -7.28785455e-11  2.91832430e-16
  1.09966374e-16]
Asociado al autovalor -15.000000003519965


## Método de la potencia inversa
Sea $A \in \mathcal{M}_n(\mathbb{C})$ invertible y sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = A^{-1}x_k \Leftrightarrow Ay_{k+1} = x_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $A^{-1}$ o, equivalentemente, el autovalor de menor módulo de $A$.

In [5]:
def potencia_inversa(A, x0, iteraciones = 100):
    tolerancia = 1e-15
    x = x0
    
    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = solve(A, x)
        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia inversa al problema para hallar el autovector asociado al autovalor de menor módulo.

In [6]:
x0 = array([1, 0, 0, 0, 0])
x = potencia_inversa(A, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[ 1.29495032e-16  4.32110488e-16  6.15248593e-16 -4.47213595e-01
 -8.94427191e-01]
Asociado al autovalor -3.000000000000001


## Método de la potencia desplazada
Sea $A \in \mathcal{M}_n(\mathbb{C})$, sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera y sea $\mu \in \mathbb{C}$.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = (A - \mu I)x_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $A - \mu I$ o, equivalentemente, el autuvalor más lejano de $\mu$.

In [7]:
def potencia_desplazada(A, mu, x0, iteraciones = 100):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None
    tolerancia = 1e-15
    x = x0

    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = (A - mu*eye(n))@x
        x = y/norm(y)

        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia inversa desplazada al problema para hallar el autovector asociado al autovalor más lejano a -5.

In [8]:
x0 = array([1, 0, 0, 0, 0])
x = potencia_desplazada(A, -5, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[-4.08248290e-01 -8.16496581e-01 -4.08248290e-01 -9.09640070e-17
  3.60735248e-18]
Asociado al autovalor 11.999999999999986


## Método de la potencia inversa desplazada
Sea $A \in \mathcal{M}_n(\mathbb{C})$, sea $x_0 \in \mathbb{C}^n \setminus \{0\}$ unitario cualquiera y sea $\mu \notin sp(A)$.
Para $k = 0, 1, 2, \dots$, hacemos
\begin{align*}
    y_{k+1} & = (A - \mu I)^{-1}x_k \Leftrightarrow (A - \mu I)y_{k+1} = x_k, \\
    x_{k+1} & = \frac{y_{k+1}}{\|y_{k+1}\|}.
\end{align*}

Bajo ciertas hipótesis, la sucesión $\{x_k\}$ converge hacia el autovector asociado al autovalor de mayor módulo de $(A - \mu I)^{-1}$ o, equivalentemente, el autovalor más cercano a $\mu$.

In [9]:
def potencia_inversa_desplazada(A, mu, x0, iteraciones = 100):
    m, n = shape(A)
    if m != n:
        print("La matriz no es cuadrada.")
        return None
    tolerancia = 1e-15
    x = x0

    iteracion = 0
    error = 1
    norma_anterior = 0

    while iteracion < iteraciones and error >= tolerancia:
        y = solve(A - mu*eye(n), x)
        norma = norm(y)
        x = y/norma
        error = abs(norma - norma_anterior)

        norma_anterior = norma
        iteracion += 1

    return x

Aplicamos el método de la potencia inversa desplazada al problema para hallar el autovector asociado al autovalor más cercano a 7.

In [10]:
x0 = array([1, 0, 0, 0, 0])
x = potencia_inversa_desplazada(A, 7, x0)
print(x)

autovalor = transpose(x)@A@x/(transpose(x)@x)
print("Asociado al autovalor", autovalor)

[-9.27912717e-16 -1.86984930e-15  4.08248290e-01  8.16496581e-01
  4.08248290e-01]
Asociado al autovalor 6.000000000000009
